In [1]:
import os

In [2]:
print('h')

h


In [3]:
%pwd

'd:\\ChestCancer\\MLOps\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\ChestCancer\\MLOps'

In [6]:
from dotenv import load_dotenv
load_dotenv()

import mlflow

with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("accuracy", 0.92)


In [7]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/yashviiishah/MLOps.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "yashviiishah"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "yb3e8c0fd7657a83d2a5bb71aa0cab0fa4aa7a41c"


In [9]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yashviiishah/MLOps.mlflow/"
os.environ["MLFLOW_TRACKING_USERNAME"]="yashviishah"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b3e8c0fd7657a83d2a5bb71aa0cab0fa4aa7a41c"


In [7]:
import mlflow

mlflow.set_tracking_uri("https://dagshub.com/yashviiishah/MLOps.mlflow/")


In [10]:
import tensorflow as tf

In [11]:

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data",
            mlflow_uri="https://dagshub.com/yashviiishah/MLOps.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-04-15 07:55:32,755: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-15 07:55:32,765: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-15 07:55:32,771: INFO: common: created directory at: artifacts]
Found 144 images belonging to 2 classes.
9/9 [==============================] - 10s 932ms/step - loss: 0.2379 - accuracy: 0.9792
[2025-04-15 07:55:43,498: INFO: common: json file saved at: scores.json]


2025/04/15 07:55:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-04-15 07:55:46,822: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\yashv\AppData\Local\Temp\tmpkapna78i\model\data\model\assets
[2025-04-15 07:55:47,777: INFO: builder_impl: Assets written to: C:\Users\yashv\AppData\Local\Temp\tmpkapna78i\model\data\model\assets]


c:\Users\yashv\anaconda3\envs\apdproj\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/04/15 07:56:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [16]:
import os

print("USERNAME:", os.getenv("MLFLOW_TRACKING_USERNAME"))
print("PASSWORD:", os.getenv("MLFLOW_TRACKING_PASSWORD"))


USERNAME: yashviishah
PASSWORD: 3231a18250cb655d859be9f158ddaf2e3716ab82


In [17]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: https://dagshub.com/yashviiishah/MLOps.mlflow/
